In [ ]:
!pip install /kaggle/input/timmwhl/timm-0.1.18-py3-none-any.whl

import cv2
from tqdm import tqdm_notebook as tqdm
import fastai
from fastai.vision import *
import os
from mish_activation import *
import warnings
warnings.filterwarnings("ignore")
import skimage.io
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from PIL import Image
from tqdm import tqdm
from skimage.transform import resize, rescale
from PIL import Image, ImageSequence
import openslide

import sys
package_path = '/kaggle/input/efficientnet-pytorch/EfficientNet-PyTorch/EfficientNet-PyTorch-master'
sys.path.append(package_path)
sys.path.insert(0, '../input/semisupervised-imagenet-models/semi-supervised-ImageNet1K-models-master/')
from hubconf import *

from efficientnet_pytorch import EfficientNet
import torch.nn as nn
import torch
import torchvision.models as models
from torch.nn import TransformerEncoder, TransformerEncoderLayer
import torch.nn.functional as F

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("device:",device)

In [ ]:

package_path = '/kaggle/input/efficientnet-pytorch/EfficientNet-PyTorch/EfficientNet-PyTorch-master'
sys.path.append(package_path)
from efficientnet_pytorch import EfficientNet

In [ ]:
##first ensemble CM
# path: /home/rvslight/pycharm/prostate/Input//LB91_224tile_best_resnext50_X20_30e_0.pth
# Predicted    0    1    2    3    4    5
# Actual                                 
# 0          658   34    0    4    2    0
# 1           28  536   23    5    3    1
# 2            1   92  176   23   14    1
# 3            8    9   46  126   52   25
# 4            3   10   16   36  176   29
# 5            7    3    6   22   36  168
#               precision    recall  f1-score   support

#            0       0.93      0.94      0.94       698
#            1       0.78      0.90      0.84       596
#            2       0.66      0.57      0.61       307
#            3       0.58      0.47      0.52       266
#            4       0.62      0.65      0.64       270
#            5       0.75      0.69      0.72       242

#     accuracy                           0.77      2379
#    macro avg       0.72      0.71      0.71      2379
# weighted avg       0.77      0.77      0.77      2379

# kappa_score 0.8984666622351744


## second ensemble CM
# path: /home/rvslight/pycharm/prostate/Input//cv8899_expand_288_tiles_resnext50_X20_30e_0.pth
# Predicted    0    1    2    3    4    5
# Actual                                 
# 0          662   30    0    3    3    0
# 1           34  528   25    6    3    0
# 2            3  121  124   38   19    2
# 3            8   17   30  117   75   19
# 4            3   18   11   43  163   32
# 5            8    1    3   16   63  151
#               precision    recall  f1-score   support

#            0       0.92      0.95      0.94       698
#            1       0.74      0.89      0.81       596
#            2       0.64      0.40      0.50       307
#            3       0.52      0.44      0.48       266
#            4       0.50      0.60      0.55       270
#            5       0.74      0.62      0.68       242

#     accuracy                           0.73      2379
#    macro avg       0.68      0.65      0.66      2379
# weighted avg       0.73      0.73      0.72      2379

# kappa_score 0.889748198293697

### third efficientnet-b2 ### cv 8822

In [ ]:
############ ensemble_list ###################

####first######
#224 tile mean / std
mean_224 = torch.tensor([1.0-0.82097102, 1.0-0.63302738, 1.0-0.75392824])
std_224 = torch.tensor([0.37723779, 0.49839178, 0.4015415])
first_resnext_pth_path_224 = "../input/lb91-224tile/LB91_224tile_best_resnext50_X20_30e_0.pth"

ensemble_1 ={"mean":mean_224,"std":std_224,"tileSize":224,"isExpandTile":False,"arch":"resnext50_32x4d","path":first_resnext_pth_path_224,"NN":64}

######second#####
#288 tile mean / std
mean_288 = torch.tensor([1.0-0.82809473, 1.0-0.65345352, 1.0-0.76934528])
std_288 = torch.tensor([0.40728608, 0.52557509, 0.42674383])
second_resnext_pth_path_288 = "../input/cv8899-expand-288/cv8899_expand_288_tiles_resnext50_X20_30e_0.pth"
expand_sz = 288

ensemble_2 ={"mean":mean_288,"std":std_288,"tileSize":288,"isExpandTile":True,"arch":"resnext50_32x4d","path":second_resnext_pth_path_288,"NN":64}


#################################################

ensemble_list = []
ensemble_list.append(ensemble_1)
ensemble_list.append(ensemble_2)


MODELS = []
for model_path_index in range(len(ensemble_list)):
    MODELS.append(ensemble_list[model_path_index]['path'])

DATA = '../input/prostate-cancer-grade-assessment/test_images'
TEST = '../input/prostate-cancer-grade-assessment/test.csv'

DATA2 = '../input/prostate-cancer-grade-assessment/train_images'
TEST2 = '../input/prostate-cancer-grade-assessment/train.csv'

SAMPLE = '../input/prostate-cancer-grade-assessment/sample_submission.csv'

mixnet_pth="../input/segmmixnetv2/checkpoint_segm_mixnet_hardaug_14elr0005_acc88.59.pth"

bs = 2
sz2=224
NUM=20 #16
NN=64 #64 #50 #60 # jjshadow.. 60-> 0.89, 50-> ??

# X20: 64-> 0.91, 68->0.89   X16: 72->0.88   #   64->0.89 (max)   #   56->0.89   #48->0.88
nworkers = 2

debug = False



In [ ]:
if os.path.exists(DATA):
    print('test set exist')
    debug = False # always off debug
    chk=False
else:
    print('test set not exist')
    chk=True
    TEST=TEST2
    DATA=DATA2
    nchk=10 #30 #100 #50 #3000 #101#3000 #3000 #1000 #100# 50
    if debug:
        nchk = 10

In [ ]:
from torch.utils.data import Dataset
from torchvision.transforms import Normalize
from torchvision import datasets, transforms#, models

def _resnext(url, block, layers, pretrained, progress, **kwargs):
    model = ResNet(block, layers, **kwargs)
    return model
    
class Model(nn.Module):
    def __init__(self, arch, n=6, pre=True):
        super().__init__()
        self.arch = arch
        if arch == 'resnet18':
            m = models.resnet18(pretrained=False)
        elif arch == 'resnet34':
            m = models.resnet34(pretrained=False)
        elif arch == 'resnet50':
            m = models.resnet50(pretrained=False)
        elif arch.startswith('efficient'):
            m = EfficientNet.from_name(arch)
            self.m = m
        else: ## resnext50_32x4d
            
            m = _resnext(semi_supervised_model_urls[arch], Bottleneck, [3, 4, 6, 3], False, 
                progress=False,groups=32,width_per_group=4)
        print("arch",arch)
        
        if arch.startswith('efficient'):
            nc = self.m._fc.in_features
            self.m._fc = nn.Identity()
            #
            avgpool = 'avgpool' if hasattr(self.m, 'avgpool') else '_avg_pooling'
            setattr(self.m, avgpool, AdaptiveConcatPool2d())

            final = nn.Linear(nc * 2, n)
            self.head = nn.Sequential(*list(self.m.children())[-4:], Flatten(), final)
        else:
            self.enc = nn.Sequential(*list(m.children())[:-2])       
            nc = list(m.children())[-1].in_features
            self.head = nn.Sequential(AdaptiveConcatPool2d(),Flatten(),nn.Linear(2*nc,512),
                    Mish(),nn.BatchNorm1d(512),nn.Dropout(0.5),nn.Linear(512,n))
        
    def forward(self, x):
        shape = x.shape
        n = shape[1]
        x = x.view(-1,shape[2],shape[3],shape[4])
        if self.arch.startswith('efficient'):
            x = self.m.extract_features(x)
        else:
            x = self.enc(x)
        shape = x.shape
        x = x.view(-1,n,shape[1],shape[2],shape[3]).permute(0,2,1,3,4).contiguous()\
          .view(-1,shape[1],shape[2]*n,shape[3])
        x = self.head(x)
        return x


class Unnormalize:
    """Converts an image tensor that was previously Normalize'd
    back to an image with pixels in the range [0, 1]."""
    def __init__(self, mean, std):
        self.mean = mean
        self.std = std

    def __call__(self, tensor):
        mean = torch.as_tensor(self.mean, dtype=tensor.dtype, device=tensor.device).view(3, 1, 1)
        std = torch.as_tensor(self.std, dtype=tensor.dtype, device=tensor.device).view(3, 1, 1)
        return torch.clamp(tensor*std + mean, 0., 1.)

normalize_transform = Normalize(mean_224, std_224)
unnormalize_transform = Unnormalize(mean_224, std_224)

def tileSegm(img,N=NN,sz=56): #N=36
    shape = img.shape
    pad0,pad1 = (sz - shape[0]%sz)%sz, (sz - shape[1]%sz)%sz
    img = np.pad(img,[[pad0//2,pad0-pad0//2],[pad1//2,pad1-pad1//2],[0,0]],
                 constant_values=255)
    img = img.reshape(img.shape[0]//sz,sz,img.shape[1]//sz,sz,3)
    img = img.transpose(0,2,1,3,4).reshape(-1,sz,sz,3)
    if len(img) < N:
        img = np.pad(img,[[0,N-len(img)],[0,0],[0,0],[0,0]],constant_values=255)
     
       
    idxs = np.argsort(img.reshape(img.shape[0],-1).sum(-1))[:N]
    img = img[idxs]
    return img, idxs

class PandaSegmDataset(Dataset):
     
    def __init__(self, tiles):
        self.tiles = tiles
        print("tiles.shape=",self.tiles.shape)
         
        
 
    def __getitem__(self, index): 
        
        tile=self.tiles[index]
        tile=torch.tensor(tile).permute((2, 0, 1)).float().div(255)
        tile= normalize_transform(tile)   
        return tile 
       
         
    def __len__(self):
        return len(self.tiles)

def predict(loader,model):
    pred =[]
    with tqdm(total=len(loader), leave=False) as pbar:
        for batch_idx, data in enumerate(loader):
             with torch.no_grad():
                x = data.to(device)
                y_pred = model(x)
                y_pred=torch.sigmoid(y_pred.squeeze())
                pred.append(y_pred.cpu().numpy() )
             pbar.update()
    pred=np.hstack(pred)            
    return pred 

def tile(img,idxs,sz=224,N=NN):
    if debug:
        print('tile_aksell')
        print("tile_sz",sz)
        print("NN",N)
    shape = img.shape
    pad0,pad1 = (sz - shape[0]%sz)%sz, (sz - shape[1]%sz)%sz
    img = np.pad(img,[[pad0//2,pad0-pad0//2],[pad1//2,pad1-pad1//2],[0,0]],
                 constant_values=255)
    img = img.reshape(img.shape[0]//sz,sz,img.shape[1]//sz,sz,3)
    img = img.transpose(0,2,1,3,4).reshape(-1,sz,sz,3)
    if len(img) < N:
        img = np.pad(img,[[0,N-len(img)],[0,0],[0,0],[0,0]],constant_values=255)   
    img = img[idxs]
    return img

def expand_tile(img,idxs,sz=224,N=NN, expand_sz = expand_sz):  # expand tile as 288 with 224 base size
    if debug:
        print('tile_expand_tile')
        print("tile_sz",expand_sz)
        print("NN",N)
    
    gap =(expand_sz - sz)
    
    shape = img.shape
    
    pad0,pad1 = (sz - shape[0]%sz)%sz, (sz - shape[1]%sz)%sz
    padgap0,padgap1 = pad0 + gap, pad1 + gap
    
    img_org = np.pad(img,[[pad0//2,pad0-pad0//2],[pad1//2,pad1-pad1//2],[0,0]],
                 constant_values=255)
    img_org_pad = np.pad(img,[[padgap0//2,padgap0-padgap0//2],[padgap1//2,padgap1-padgap1//2],[0,0]],
                 constant_values=255)
    
    img_w_h = img_org.reshape(img_org.shape[0]//sz,sz,img_org.shape[1]//sz,sz,3)
    img_w_h = img_w_h.transpose(0,2,1,3,4)
    img = img_w_h.reshape(-1,sz,sz,3)
    
    expand_img_list = []
    
    for vertical_row in range(0,img_w_h.shape[0]):
        for horizontal_row in range(0,img_w_h.shape[1]):
            expand_tile_img = img_org_pad[ (sz*vertical_row):(sz*(vertical_row+1))+gap , (sz*horizontal_row):(sz*(horizontal_row+1))+gap]
            expand_img_list.append(expand_tile_img) 
    
    expand_img = np.asarray(expand_img_list)
    
    if debug: #debug
        print("gap",gap)
        print('img_w_h',img_w_h.shape)
        print("img.shape",img.shape)
        print('expand_img_shape',expand_img.shape)
    
    if len(expand_img) < N:
        expand_img = np.pad(expand_img,[[0,N-len(expand_img)],[0,0],[0,0],[0,0]],constant_values=255)  
    expand_img = expand_img[idxs]
    return expand_img

def loadANDtile(image_id,tile_idx, N=NN, phase=0):
    level=1
    img = skimage.io.MultiImage(os.path.join(DATA,image_id+'.tiff'))[level] # mid res

    isExpandTile = ensemble_list[phase]['isExpandTile']
    if debug:
        print("isExpandTile",isExpandTile)
        print("NN",NN)
    if isExpandTile:
        tiles=expand_tile(img,tile_idx, N=N, expand_sz = ensemble_list[phase]['tileSize'])
    else:
        tiles=tile(img,tile_idx, N=N)
    return tiles    

class ResnetDataset(Dataset):
    def __init__(self, df, N=NN, phase=0):
       # self.path = path
        self.df = df
        self.N = N
        self.phase = phase
        print("length=",len(self.df))
         
        
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        image_id = row["image_id"] 
        tile_idx = row["tile_idx"] 
        tile_idx=[int(i) for i in tile_idx]
        tiles=loadANDtile(image_id,tile_idx, N=self.N, phase=self.phase)
        tiles = torch.Tensor(1.0 - tiles/255.0)
        
        mean = ensemble_list[self.phase]['mean']
        std = ensemble_list[self.phase]['std']
        if debug:
            print("mean",mean)
            print("std",std)
        tiles = (tiles - mean)/std
        return tiles.permute(0,3,1,2), image_id

In [ ]:
import timm
segmmodel = timm.create_model('mixnet_xl', pretrained=False)
for param in segmmodel.parameters():
    param.requires_grad = False
    # Replace the last fully-connected layer
    # Parameters of newly constructed modules have requires_grad=True by default
segmmodel.classifier=nn.Linear(1536, 1)
segmmodel.fc = nn.Linear(1536, 1)

In [ ]:
checkpoint = torch.load(mixnet_pth, map_location=device)
segmmodel.load_state_dict(checkpoint)
segmmodel.eval()
segmmodel.cuda() 
del checkpoint

In [ ]:
aknell_models = []
for model_index, path in enumerate(MODELS):
    #
    print("path",path)
    state_dict = torch.load(path,map_location=torch.device(device))   
    model = Model(arch=ensemble_list[model_index]['arch'])
    model.load_state_dict(state_dict)
    model.float()
    model.eval()
    model.cuda()   
    aknell_models.append(model)

del state_dict

In [ ]:
test=pd.read_csv(TEST)
if chk:
    pass
    test=test[:][:nchk]

In [ ]:
names=[]
Aksell_PRED_LIST=[]
for pred_list_index in range(len(ensemble_list)):
    Aksell_PRED_LIST.append([])

for index, ensemble_parameter in enumerate(ensemble_list):
    NN = ensemble_parameter['NN']
    print("Current_MODEL_PATH",str(index),ensemble_parameter['path'])
    print("Current_NN",str(index),NN)
    
    imid=[]
    idxs=[]
    tiles=[]
    level=2 #low res
    for i in tqdm(range(len(test))):
        img = skimage.io.MultiImage(os.path.join(DATA,test.iloc[i].image_id+'.tiff'))[level]
        ti,idx=tileSegm(img, N=NN)
        imid.append(test.iloc[i].image_id)
        idxs.append(idx)
        tiles.append(ti)

    tiles= np.array(tiles)
    tiles=tiles.reshape(tiles.shape[0]*tiles.shape[1],tiles.shape[2],tiles.shape[3],3  )
    idxs=np.array(idxs).flatten()
    imid =np.repeat(imid, NN) 

    dataset = PandaSegmDataset(tiles)
    loader = DataLoader(dataset, batch_size=128, shuffle=False, num_workers=2, pin_memory=True)
    
    with torch.no_grad():
        y=predict(loader,segmmodel)

    df=pd.DataFrame({ 'image_id':imid ,  'idxs':idxs,'pred':y}) 
    df=df.sort_values(["image_id","pred"],ascending=False)
    df['oq'] = df.groupby('image_id').cumcount() + 1
    df=df.loc[df.oq<=NUM]

    test=test.set_index('image_id')
    test["tile_idx"]=np.nan
    test=test.astype(object)

    iuniq=df.image_id.unique()
    for i in tqdm(iuniq):
        idxs=df.loc[df.image_id==i].idxs.values
        test.at[i,"tile_idx"]=idxs
    test = test.reset_index()
    del dataset
    del loader
    del df
    import gc
    gc.collect()

    ds = ResnetDataset(test, N=NN, phase=index)

    bs = 2
    dl = DataLoader(ds, batch_size=bs, num_workers=nworkers, shuffle=False)

    
    with torch.no_grad():
        
        sz2 = ensemble_parameter['tileSize']
        print("tilsSize:",sz2)
        print(f"{index} prediction with NN:{NN}")
        for x,y in dl:#loader1:    

            x = x.to(device)
            #dihedral TTA

            x = torch.stack([x,x.flip(-1),x.flip(-2),x.flip(-1,-2),
              x.transpose(-1,-2),x.transpose(-1,-2).flip(-1),
              x.transpose(-1,-2).flip(-2),x.transpose(-1,-2).flip(-1,-2)],1)
            x = x.view(-1,NUM,3,sz2,sz2)
            p = aknell_models[index](x) 
            pred_mean=p.view(bs,8,-1).mean(1) 
            Aksell_PRED_LIST[index].extend(pred_mean.argmax(-1).detach().cpu().numpy())
            
            del x,p
            torch.cuda.empty_cache()
            if index == 0: ## just first case add y
                names.append(y)
if debug:             
    for check_p_index in range(len(Aksell_PRED_LIST)):
        print(len(Aksell_PRED_LIST[check_p_index]))
print("at last concat phase")
names = np.concatenate(names)
# for pred_value_index, unit_pred in enumerate(Aksell_PRED_LIST):
#     if pred_value_index == 0:
#         total = (torch.cat(PRED_LIST[pred_value_index]).sigmoid().cpu())
#     else:
#         total += (torch.cat(PRED_LIST[pred_value_index]).sigmoid().cpu())

# LOGITS = total / len(PRED_LIST)
# preds=LOGITS.argmax(-1).numpy()


In [ ]:
# ce_preds = [float(x) for x in preds] # to float for ensemble with regression

In [ ]:
####### shujun code merge ##########

test_image_dir='/kaggle/input/prostate-cancer-grade-assessment/test_images'
if os.path.exists(test_image_dir):
    print('test set exist')
    chk=False
    mode='test'
    test_image_dir='/kaggle/input/prostate-cancer-grade-assessment/{}_images'.format(mode)
    csv_path='/kaggle/input/prostate-cancer-grade-assessment/{}.csv'.format(mode)
else: #before test debug with 100 train image
    print('test set not exist')
    chk=True    
    mode='train'
    test_image_dir='/kaggle/input/prostate-cancer-grade-assessment/{}_images'.format(mode)
    csv_path='/kaggle/input/prostate-cancer-grade-assessment/{}.csv'.format(mode)
    
submission_path='/kaggle/input/prostate-cancer-grade-assessment/sample_submission.csv'
    
df=pd.read_csv(csv_path)
if chk:
    pass
    df=df[:][:nchk]

##################### shujun ensemble recipe ###########################

#### ensemlbe #1 36x256x256 resnext50_32x4d arch lb: 0.911 best one ######
    
mean_256 = [0.85841531, 0.78983734, 0.89534979] # original shujun 256 cut
std_256 = [0.16897951, 0.25758676, 0.13730845] # original shujun 256 cut
first_effb3_path_256 = '/kaggle/input/pandalayertest4wunertaintyacc/pandalayertest4wunertaintyacc/fold{}top{}.ckpt'

ensemble_shujun_1 ={"mean":mean_256,"std":std_256,"tileImageSize":256,"isCutThreshold":False,"arch":"resnext50_32x4d","path":first_effb3_path_256,"TILE_SIZE":36,"TILE_ALL":False,"CVT_COLOR":True}

#### ensemble #2 29x224x224 resnext50_32x4d arch qwk: 0.8996019660786133 ######

mean_224= [0.81702482, 0.6299724, 0.75427603] # 224 cut
std_224=[0.40702703, 0.52099041, 0.42290275] # 224 cut
second_resnext50_path_224 = '/kaggle/input/pandaakselltest3/fold{}top{}.ckpt'

ensemble_shujun_2 ={"mean":mean_224,"std":std_224,"tileImageSize":224,"isCutThreshold":True,"arch":"resnext50_32x4d","path":second_resnext50_path_224,"TILE_SIZE":20,"TILE_ALL":False,"CVT_COLOR":True}

#### ensemble #3 30x224x224 resnext50_32x4d  ######
mean_224_30tile= [0.82514849, 0.64166622, 0.76170417] # 224 cut
std_224_30tile=[0.40444484, 0.52074353, 0.42308628] # 224 cut
third_resnext50_path_224 = '/kaggle/input/pandaakselltest4/fold{}top{}.ckpt'
ensemble_shujun_3 ={"mean":mean_224_30tile,"std":std_224_30tile,"tileImageSize":224,"isCutThreshold":True,"arch":"resnext50_32x4d","path":third_resnext50_path_224,"TILE_SIZE":30,"TILE_ALL":False,"CVT_COLOR":False}

# #### ensemble #4 20x288x288 resnext50_32x4d  ######
# mean_288= [0.82809473, 0.65345352, 0.76934528] # 288 cut
# std_288=[0.40728608, 0.52557509, 0.42674383] # 288 cut
mean_288 = mean_224
std_288 = std_224
fourth_resnext50_path_288 = '/kaggle/input/pandaakselltest5/fold{}top{}.ckpt'
ensemble_shujun_4 ={"mean":mean_288,"std":std_288,"tileImageSize":288,"isCutThreshold":True,"arch":"resnext50_32x4d","path":fourth_resnext50_path_288,"TILE_SIZE":20,"TILE_ALL":False,"CVT_COLOR":False}
## no cvtColor

##### snsemble #5 32x224x224 ####
mean_224_32= [0.82514849, 0.64166622, 0.76170417] # 224 32
std_224_32=[0.40444484, 0.52074353, 0.42308628] # 224 32
fifth_resnext50_path_224 = '/kaggle/input/pandaakselltest7/fold{}top{}.ckpt'
ensemble_shujun_5 ={"mean":mean_224_32,"std":std_224_32,"tileImageSize":224,"isCutThreshold":True,"arch":"resnext50_32x4d","path":fifth_resnext50_path_224,"TILE_SIZE":32,"TILE_ALL":True,"CVT_COLOR":True}
## no cvtColor

##### janey's model ensemble #6 64x224x224 ####
mean_256_64= [0.8021112084388733, 0.7040286064147949, 0.8552106022834778] # 224 32
std_256_64=[0.18155023455619812, 0.27437373995780945, 0.15957728028297424] # 224 32
sixth_resnext50_path_256 = '/kaggle/input/resnext50-new/fold{}top{}.ckpt'
ensemble_shujun_6 ={"mean":mean_256_64,"std":std_256_64,"tileImageSize":256,"isCutThreshold":False,"arch":"resnext50_32x4d","path":sixth_resnext50_path_256,"TILE_SIZE":64,"TILE_ALL":False,"CVT_COLOR":True}


###########################################################################
## remove 30x224x224

ensemble_shujun_list = []
ensemble_shujun_list.append(ensemble_shujun_1)
ensemble_shujun_list.append(ensemble_shujun_2)
ensemble_shujun_list.append(ensemble_shujun_4)
ensemble_shujun_list.append(ensemble_shujun_5)
ensemble_shujun_list.append(ensemble_shujun_6)

In [ ]:
### shujun common structure ###
# mish activation
class Mish(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        #inlining this saves 1 second per epoch (V100 GPU) vs having a temp x and then returning x(!)
        return x *( torch.tanh(F.softplus(x)))


from torch.nn.parameter import Parameter
def gem(x, p=3, eps=1e-6):
    return F.avg_pool2d(x.clamp(min=eps).pow(p), (x.size(-2), x.size(-1))).pow(1./p)

class GeM(nn.Module):
    def __init__(self, p=3, eps=1e-6):
        super(GeM,self).__init__()
        self.p = Parameter(torch.ones(1)*p)
        self.eps = eps
    def forward(self, x):
        return gem(x, p=self.p, eps=self.eps)
    def __repr__(self):
        return self.__class__.__name__ + '(' + 'p=' + '{:.4f}'.format(self.p.data.tolist()[0]) + ', ' + 'eps=' + str(self.eps) + ')'

class MultiheadAttentionClassifier(nn.Module):
    def __init__(self,num_classes,out_features,ninp,nhead,dropout,attention_dropout=0.1):
        super(MultiheadAttentionClassifier, self).__init__()
        self.attention=nn.MultiheadAttention(ninp, nhead, dropout=attention_dropout)
        self.classifier=nn.Linear(ninp*2,num_classes)
        self.dropout=nn.Dropout(dropout)
        self.mish=Mish()

    def forward(self,x):
        x=x.permute(1,0,2)
        x,_=self.attention(x,x,x)
        x=self.mish(x)
        x=x.permute(1,0,2)
        max_x,_=torch.max(x,dim=1)
        x=torch.cat([torch.mean(x,dim=1),max_x],dim=-1)
        x=self.dropout(x)
        x=self.classifier(x)
        return x

def _resnext(url, block, layers, pretrained, progress, **kwargs):
    model = ResNet(block, layers, **kwargs)
    return model

class Network(nn.Module):
    def __init__(self,num_classes,arch='efficientnet-b0',dropout_p=0.5,nhead=8,ninp=512, nhid=512,attention_dropout=0.1):
        super(Network, self).__init__()
        if debug:
            print('arch',arch)
        if arch.startswith('efficient'):
            self.backbone=EfficientNet.from_name(arch)
        elif arch == 'resnet34':
            self.backbone=models.resnet34(pretrained=False,)
        elif arch == 'resnext50_32x4d':
            self.backbone=models.resnext50_32x4d(pretrained=False)
        self.backbone.avgpool=GeM()
        self.num_classes=num_classes
        self.out_features = self.backbone.fc.in_features
        self.ninp=ninp
        self.backbone.fc=nn.Linear(self.out_features,ninp)
        self.regression_head=MultiheadAttentionClassifier(1,self.out_features,ninp,nhead,dropout_p)
        self.classifier_head=MultiheadAttentionClassifier(num_classes[0],self.out_features,ninp,nhead,dropout_p)
        self.regression_head2=MultiheadAttentionClassifier(1,self.out_features,ninp,nhead,dropout_p)
        self.regression_head3=MultiheadAttentionClassifier(1,self.out_features,ninp,nhead,dropout_p)

    def forward(self,x):
        shape = x[0].shape
        bs=x.shape[0]
        n = len(x)
        x = x.reshape((-1,shape[1],shape[2],shape[3]))
        features=self.backbone(x)
        features=features.reshape(bs,shape[0],self.ninp)
        outputs=[self.regression_head(features).squeeze(),F.log_softmax(self.classifier_head(features),dim=1),
                 self.regression_head2(features).squeeze(),self.regression_head3(features).squeeze()]
        
        return outputs
def tile_shujun(img, sz=224,N=20):
    if debug:
        print('tile_shujun')
    result = []
    shape = img.shape
    pad0,pad1 = (sz - shape[0]%sz)%sz, (sz - shape[1]%sz)%sz
    img = np.pad(img,[[pad0//2,pad0-pad0//2],[pad1//2,pad1-pad1//2],[0,0]],
                constant_values=255)
    img = img.reshape(img.shape[0]//sz,sz,img.shape[1]//sz,sz,3)
    img = img.transpose(0,2,1,3,4).reshape(-1,sz,sz,3)
    if len(img) < N:
        img = np.pad(img,[[0,N-len(img)],[0,0],[0,0],[0,0]],constant_values=255)
    idxs = np.argsort(img.reshape(img.shape[0],-1).sum(-1))[:N]
    img = img[idxs]
    return img

def tile_all(img, N, sz=256):
    if debug:
        print('tile_all_shujun')
        print('N',N)
    result = []
    shape = img.shape
    pad0,pad1 = (sz - shape[0]%sz)%sz, (sz - shape[1]%sz)%sz
    img = np.pad(img,[[pad0//2,pad0-pad0//2],[pad1//2,pad1-pad1//2],[0,0]],
                constant_values=255)
    img = img.reshape(img.shape[0]//sz,sz,img.shape[1]//sz,sz,3)
    img = img.transpose(0,2,1,3,4).reshape(-1,sz,sz,3)
    if len(img) < N:
         img = np.pad(img,[[0,N-len(img)],[0,0],[0,0],[0,0]],constant_values=255)
    idxs = np.argsort(img.reshape(img.shape[0],-1).sum(-1))
    
    return img, idxs

def standardize_batch(batch,mean,std,isCutThresholdTile):
    ##1. change standardize_batch
    if debug:
        print("test:",batch.shape[1])
    if batch.shape[1]==64:
        if debug:
            print("st_01")
        batch=batch/255.0
        batch=(batch-mean)/std
    elif isCutThresholdTile:
        if debug:
            print("st_02")
        batch=batch/255.0
        batch=(mean-batch)/std
    else:
        if debug:
            print("st_03")
        batch=batch/255.0
    return batch

def get_batch_of_image(paths, sz, N, phase = 0, index_df = None):
    isCutThreshold = ensemble_shujun_list[phase]['isCutThreshold']
    isTileAll = ensemble_shujun_list[phase]['TILE_ALL']
    if debug:
        print('sz',sz)
        print('N',N)
        print('isCutThreshold',isCutThreshold)
    images=[]
    for path in paths:
        if isCutThreshold:
            tile_idx = index_df.loc[index_df['image_id']== path]['tile_idx'].tolist()[0]
            tile_idx=[int(i) for i in tile_idx]
            if debug:
                print(tile_idx)
                
        path=os.path.join(test_image_dir,path+'.tiff')
        img = skimage.io.MultiImage(path,conserve_memory=False,dtype='tiff')[1]
        if isCutThreshold:
            
            if isTileAll:
                ## first just get 20 aknell tile
                tile_idx = tile_idx[:20]
                if debug:
                    print(tile_idx)
                Max_Indices_N = max(tile_idx)+1
                if debug:
                    print("Max_Indices_N",Max_Indices_N)
                img, idxs = tile_all(img, sz=sz, N=Max_Indices_N)
                tiles = []
                for sorted_index in idxs:
                    if sorted_index not in tile_idx:
                        tiles.append(img[sorted_index])
                    if len(tiles)>=N-20:
                        break
                ## 5. add padding function for combine tile models:
                if len(tiles) < N-20:
                    tiles = np.pad(tiles,[[0,N-len(tiles)],[0,0],[0,0],[0,0]],constant_values=255)
                tiles = np.asarray(tiles)
                if debug:
                    print("before_concate_tiles_shape",tiles.shape)
                    print("aksell_indices.shape",img[tile_idx].shape)
                tiles = np.concatenate([tiles,img[tile_idx]]).astype("uint8")
                if debug:
                    print("tiles",tiles.shape)
                img = tiles
                        
            else:
                if sz == 224: ## laffos cut with cancer index ## janey case's isCutThreshold is False so go to tile_shujun
                    img = tile(img,tile_idx, N=NN)
                else: ## expand tile with canner index
                    img = expand_tile(img,tile_idx, sz=224, N=NN, expand_sz = sz)
                
        else:
            img = tile_shujun(img, sz=sz, N=N)
            
        if(ensemble_shujun_list[phase]['CVT_COLOR']==True): ## CVT_COLOR is False in pandatest4,test5
            if debug:
                print("CVT_COLOR is True")
            for j in range(img.shape[0]):
                img[j] = cv2.cvtColor(img[j],cv2.COLOR_RGB2BGR)
        else:
            if debug:
                print("CVT_COLOR is False")
        img=np.asarray(img,dtype='uint8')
        images.append(img)
    images=np.asarray(images,dtype='uint8').transpose((0,1,4,2,3))
    return images


def classification_threshold(tensor):
    for i in range(len(tensor)):
        if tensor[i] > 4.5:
            tensor[i]=5
        elif tensor[i] > 3.5:
            tensor[i]=4
        elif tensor[i] > 2.5:
            tensor[i]=3
        elif tensor[i] > 1.5:
            tensor[i]=2
        elif tensor[i] > 0.6:
            tensor[i]=1
        elif tensor[i] < 0.6:
            tensor[i]=0
    return tensor.astype('int')

def classification_threshold_item(item):
    if item > 4.5:
        item=5
    elif item > 3.5:
        item=4
    elif item > 2.5:
        item=3
    elif item > 1.5:
        item=2
    elif item > 0.6:
        item=1
    elif item < 0.6:
        item=0
    return int(item)#tensor.astype('int')

def hflip(tensor):
    for i in range(tensor.shape[0]):
        for j in range(tensor.shape[1]):
            tensor[i,j]=np.flip(tensor[i,j],axis=0)
    return tensor


def vflip(tensor):
    for i in range(tensor.shape[0]):
        for j in range(tensor.shape[1]):
            tensor[i,j]=np.flip(tensor[i,j],axis=1)
    return tensor

In [ ]:

import torch.nn as nn
class JaneyMultiheadAttentionClassifier(nn.Module):
    def __init__(self,num_classes,out_features,ninp,nhead,dropout,nlayers=1,attention_dropout=0.1):
        super(JaneyMultiheadAttentionClassifier, self).__init__()
        encoder_layers = nn.TransformerEncoderLayer(ninp, nhead, ninp*4, attention_dropout)
        encoder_layers.activation=Mish()
        self.attention = nn.TransformerEncoder(encoder_layers, nlayers)
        self.classifier=nn.Linear(ninp*2,num_classes)
        self.dropout=nn.Dropout(dropout)

    def forward(self,x):
        x=self.dropout(x)
        x=x.permute(1,0,2)
        x=self.attention(x)
        x=x.permute(1,0,2)
        max_x,_=torch.max(x,dim=1)
        x=torch.cat([torch.mean(x,dim=1),max_x],dim=-1)
        x=self.dropout(x)
        x=self.classifier(x)
        return x


class JaneyNetwork(nn.Module):
    def __init__(self,num_classes,efficientNet_name='efficientnet-b0',dropout_p=0.5,nhead=8,ninp=512, nhid=512,attention_dropout=0.1):
        super(JaneyNetwork, self).__init__()
#         self.backbone=torch.hub.load('facebookresearch/semi-supervised-ImageNet1K-models', 'resnext50_32x4d_ssl')
        self.backbone=models.resnext50_32x4d(pretrained=False)
        self.backbone.avgpool=GeM()
        self.num_classes=num_classes
        self.out_features = self.backbone.fc.in_features
        self.ninp=ninp
        self.backbone.fc=nn.Identity()
        regression_head=JaneyMultiheadAttentionClassifier(1,self.out_features,ninp,nhead,dropout_p)
        classifier_head=JaneyMultiheadAttentionClassifier(num_classes[0],self.out_features,ninp,nhead,dropout_p)
        regression_head2=JaneyMultiheadAttentionClassifier(1,self.out_features,ninp,nhead,dropout_p)
        regression_head3=JaneyMultiheadAttentionClassifier(1,self.out_features,ninp,nhead,dropout_p)

        self.heads=nn.ModuleList([regression_head,classifier_head,regression_head2,regression_head3,nn.Linear(self.out_features,ninp)])

    def forward(self,x):
        shape = x[0].shape
        bs=x.shape[0]
        n = len(x)
        x = x.reshape((-1,shape[1],shape[2],shape[3]))
        features=self.backbone(x)
        features=self.heads[4](features)
        features=features.reshape(bs,shape[0],self.ninp)
        outputs=[self.heads[0](features).squeeze(-1),F.log_softmax(self.heads[1](features),dim=1),
                 self.heads[2](features).squeeze(-1),self.heads[3](features).squeeze(-1)]
        return outputs


In [ ]:
### make segment model index_df for shujun ###
Max_Cut_Num = NN
if debug:
    print('Max_Cut_Num',Max_Cut_Num)

imid=[]
idxs=[]
tiles=[]
level=2 #low res
for i in tqdm(range(len(df))):
    img = skimage.io.MultiImage(os.path.join(DATA,df.iloc[i].image_id+'.tiff'))[level]
    ti,idx=tileSegm(img, N=NN)
    imid.append(df.iloc[i].image_id)
    idxs.append(idx)
    tiles.append(ti)

tiles= np.array(tiles)
tiles=tiles.reshape(tiles.shape[0]*tiles.shape[1],tiles.shape[2],tiles.shape[3],3  )
idxs=np.array(idxs).flatten()
imid =np.repeat(imid, NN) 

dataset = PandaSegmDataset(tiles)
loader = DataLoader(dataset, batch_size=128, shuffle=False, num_workers=2, pin_memory=True)

with torch.no_grad():
    y=predict(loader,segmmodel)

index_df=pd.DataFrame({ 'image_id':imid ,  'idxs':idxs,'pred':y}) 
index_df=index_df.sort_values(["image_id","pred"],ascending=False)
index_df['oq'] = index_df.groupby('image_id').cumcount() + 1
index_df=index_df.loc[index_df.oq<=20] ## maybe 64 index.. sorted

In [ ]:
### insert index to df file 
df=df.set_index('image_id')
df["tile_idx"]=np.nan
df=df.astype(object)

iuniq=index_df.image_id.unique()
for i in tqdm(iuniq):
    idxs=index_df.loc[index_df.image_id==i].idxs.values
    df.at[i,"tile_idx"]=idxs
df = df.reset_index()
del dataset
del loader
del index_df
del y
import gc
gc.collect()

In [ ]:
Shujun_PRED_LIST=[]
for pred_list_index in range(len(ensemble_shujun_list)):
    Shujun_PRED_LIST.append([])


for phase_index, ensemble_recipe in enumerate(ensemble_shujun_list):
    print("shujun_ensemble_phase",phase_index)

    MODELS=[]
    top=3
    num_classes=[6,4,4,]

    if ensemble_recipe['tileImageSize'] == 288:
        print("top_fold is 3")
        top_fold = 3 ## just 2 phase.. 4th ensemble model's top fold is 3(20x288x288)
    else:
        top_fold = 2
    for i in range(top_fold,top_fold+1):
        for j in range(top):
            
            if ensemble_recipe['TILE_SIZE'] == 64: ## janey network.
                if debug:
                    print("Janey Network")
                model = JaneyNetwork(num_classes).to(device)
            else:
                model = Network(num_classes,arch = ensemble_recipe['arch']).to(device)
            model = model.to(device)
            model = nn.DataParallel(model)
            weights_paths=ensemble_recipe['path'].format(i,j+1)
            if debug:
                print(weights_paths)
            model.load_state_dict(torch.load(weights_paths))
            model.eval()
            MODELS.append(model) 

    ####shujun code output ###

    total_images=0
    ## janey model's batch is 8 for OOM
    if ensemble_recipe['TILE_SIZE'] == 64 or ensemble_recipe['tileImageSize'] == 288: ## janey network.
        batch_size=8
    else:
        batch_size=16
    if debug:
            print("batch_size",batch_size)
    if len(df)%batch_size==0:
        batches=int(len(df)/batch_size)
    else:
        batches=int(len(df)/batch_size)+1
    model.eval()
    predictions=[]
    mean=torch.Tensor(ensemble_recipe['mean']).to(device).reshape(1,1,3,1,1)
    std=torch.Tensor(ensemble_recipe['std']).to(device).reshape(1,1,3,1,1)
    if debug:
        print(mean)
        print(std)
    with torch.no_grad():
        for i in tqdm(range(batches)):
            paths=df.image_id[i*batch_size:(i+1)*batch_size].to_list()
            x=get_batch_of_image(paths, sz = ensemble_recipe['tileImageSize'], N = ensemble_recipe['TILE_SIZE'], phase=phase_index, index_df = df)
            x0=torch.Tensor(x).to(device)
            x_h=torch.Tensor(x).to(device).flip(-1)
            x_v=torch.Tensor(x).to(device).flip(-2)
            x_vh=torch.Tensor(x).to(device).flip(-1,-2)
            x0t=torch.Tensor(x).to(device).transpose(-1,-2)
            x_ht=torch.Tensor(x).to(device).flip(-1).transpose(-1,-2)
            x_vt=torch.Tensor(x).to(device).flip(-2).transpose(-1,-2)
            x_vht=torch.Tensor(x).to(device).flip(-1,-2).transpose(-1,-2)
            outputs=[]
            isCutThresholdTile = ensemble_recipe['isCutThreshold']
            if debug:
                print('isCutThresholdTile',isCutThresholdTile)
            for model in MODELS:
                output = model(standardize_batch(x0,mean,std,isCutThresholdTile=isCutThresholdTile))[0]+\
                         model(standardize_batch(x_h,mean,std,isCutThresholdTile=isCutThresholdTile))[0]+\
                         model(standardize_batch(x_v,mean,std,isCutThresholdTile=isCutThresholdTile))[0]+\
                         model(standardize_batch(x_vh,mean,std,isCutThresholdTile=isCutThresholdTile))[0]+\
                         model(standardize_batch(x0t,mean,std,isCutThresholdTile=isCutThresholdTile))[0]+\
                         model(standardize_batch(x_ht,mean,std,isCutThresholdTile=isCutThresholdTile))[0]+\
                         model(standardize_batch(x_vt,mean,std,isCutThresholdTile=isCutThresholdTile))[0]+\
                         model(standardize_batch(x_vht,mean,std,isCutThresholdTile=isCutThresholdTile))[0]


                output=output.cpu().numpy()/8
                outputs.append(output)
            outputs=np.asarray(outputs)
            outputs=np.mean(outputs,axis=0)

            predictions.append(output)
    for pred in predictions:
        Shujun_PRED_LIST[phase_index].extend(pred)
    
    del output, predictions, MODELS ### fix OOM memory issue.. hope..
    torch.cuda.empty_cache()


#### here shujun ensemble.. ####
if debug:
    for check_p_index in range(len(Shujun_PRED_LIST)):
        print(len(Shujun_PRED_LIST[check_p_index]))
    print("at last ensemble phase")

shujun_pred_list = np.asarray(Shujun_PRED_LIST)

print("shape",shujun_pred_list.shape)
if debug:
    print(Aksell_PRED_LIST) 
    print(Shujun_PRED_LIST) 
total_list = np.vstack((np.asarray(Aksell_PRED_LIST),np.asarray(Shujun_PRED_LIST))).transpose()
#shujun_ensemble_preds = np.asarray(PRED_LIST).mean(axis=0) ## shujun ensemble here

In [ ]:
#### here ensemble.. must do it!!!
label_list = []
for total_list_index, list_item in enumerate(total_list.tolist()):
    predictions = list(list_item)
    final_value = np.mean(predictions)
    label=classification_threshold_item(final_value)
    if debug:
        print("final_value",final_value)
        print("!!label!!",label)
    label_list.append(label)
    
predictions_output=np.asarray(label_list,dtype='int')


###output###
sub_df = pd.DataFrame({'image_id':df.image_id.tolist(), 'isup_grade': list(predictions_output)})


if chk:
    from sklearn import metrics
    ground_truths=np.asarray(df.isup_grade,dtype='int')
    score=metrics.cohen_kappa_score(ground_truths,predictions_output,weights='quadratic')
    print(score)
    sub_df['ig']=df.isup_grade[:nchk]

sub_df.to_csv('submission.csv',index=False)

In [ ]:
sub_df.head(50)

In [ ]:
sub_df.tail(50)

In [ ]:
sub_df.isup_grade.unique()

In [ ]:
# from IPython.display import HTML
# import pandas as pd
# import numpy as np
# import base64

# # function that takes in a dataframe and creates a text link to  
# # download it (will only work for files < 2MB or so)
# def create_download_link(df, title = "Download CSV file", filename = "submission.csv"):  
#     csv = df.to_csv()
#     b64 = base64.b64encode(csv.encode())
#     payload = b64.decode()
#     html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
#     html = html.format(payload=payload,title=title,filename=filename)
#     return HTML(html)

In [ ]:
# create_download_link(sub_df)